In [1]:
import sys
import re
import os
from datetime import datetime
from time import sleep
from IPython.display import display, clear_output, HTML

# set working dir
WORKING_DIR = re.sub('jdi-qasp-ml.*$','',os.path.normpath(os.getcwd())) + 'jdi-qasp-ml'
os.chdir(WORKING_DIR)

import utils
from utils import *
import torch
import pandas as pd
import numpy as np
import torch
from glob import glob
import selenium
from tqdm.auto import tqdm

START_TS = datetime.now()
display(HTML(f'{START_TS} - Current working directory: <b>{WORKING_DIR}</b>'))

import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline

import numba


2021-06-10 15:25:51,323 -INFO - config:config.py:16 - Module utils.config was loaded
2021-06-10 15:25:51,326 -INFO - common:common.py:618 - Module utils.common is loaded...
2021-06-10 15:25:51,345 -INFO - hidden:hidden.py:121 - hidden module is loaded
2021-06-10 15:25:51,389 -INFO - dataset_builder:dataset_builder.py:207 - dataset package is loaded...
2021-06-10 15:26:10,438 -INFO - dataset:dataset.py:653 - dataset module is loaded...


In [3]:
# JDIDataset.gen_dataset_names()
datasets = 

{'ac-avto161',
 'angular',
 'bootstrap',
 'bootstrap-1',
 'bootstrap-form',
 'bootstrap-forms',
 'bootstrap-reboot',
 'cnews',
 'complex-table',
 'contact-form',
 'dates',
 'different-elements',
 'dummy',
 'gitlab',
 'google-voice',
 'html-5',
 'login',
 'material-ui-Button Groups',
 'material-ui-Buttons',
 'material-ui-Checkbox',
 'material-ui-Floating Action Button',
 'material-ui-List',
 'material-ui-Progress',
 'material-ui-Radio',
 'material-ui-Select',
 'material-ui-Slider',
 'material-ui-Switch',
 'material-ui-Text Field',
 'material-ui-Textarea Autosize',
 'metals-and-colors',
 'mobile-and-html-5',
 'ms-office',
 'ozon',
 'performance',
 'phys-org',
 'react-ant',
 'search',
 'simple-table',
 'support',
 'table-with-pages',
 'user-table',
 'wildberries'}